## Generate Filenames

Generate a list of filenames for a label class

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cPickle as pickle
import os

%matplotlib inline

In [2]:
BASE_DATA_PATH = '/home/griffin/projects/pulseClassifier/data/'

In [3]:
baseBufferPklFile = BASE_DATA_PATH + 'featureDateframe.pkl'

# load baseBufferPkl
df = pd.read_pickle(baseBufferPklFile)

## Select out a single label

### Labels

#### RFI

* 1 : Unclipped RFI/Noise
* 2 : Wide-band, long-duration in time clipped RFI (2016+)
* 3 : Wide-band, short-duration in time clipped RFI (2016+)
* 4 : Wide-band, short duration clipped RFI (2015)
* 5 : Sharp bandpass transition
* 6 : Wide-band, bursty clipped RFI (2015)
* 7 : Error in spectra captue or replacement
* 8 : Systematic int/float overflow

#### Astronomical

* 0 : Interesting, follow up
* 9 : Known Pulsar

In [4]:
labelID = 9

In [5]:
df.columns

Index([u'datfile', u'Beam', u'TSID', u'Buffer', u'MJDstart', u'bestDM',
       u'bestSNR', u'BinFactor', u'Events', u'DMmax',
       ...
       u'pixelsMean133', u'pixelsMean140', u'pixelsMean141', u'pixelsMean142',
       u'pixelsMean143', u'pixelsMean150', u'pixelsMean151', u'pixelsMean152',
       u'pixelsMean153', u'DMin'],
      dtype='object', length=407)

In [11]:
for col in df.columns: print col

datfile
Beam
TSID
Buffer
MJDstart
bestDM
bestSNR
BinFactor
Events
DMmax
DMmin
DMmean
DMmedian
DMstd
SNRmean
SNRmedian
SNRstd
MJDmax
MJDmin
MJDstd
MJDmean
MJDmedian
Label
predictLabel
filterbank
pctZero
pctZeroDeriv
ofCount
ofPct
longestRun0
longestRun1
longestRun2
globtsStatsStd
globtsStatsMax
globtsStatsPosCnt
globtsStatsMin
globtsStatsNegPct
globtsStatsMedian
globtsStatsRatio0
globtsStatsPosPct
globtsStatsNegCnt
globtsStatsRatio1
globtsStatsMean
globDedisptsStatsStd
globDedisptsStatsMax
globDedisptsStatsPosCnt
globDedisptsStatsMin
globDedisptsStatsNegPct
globDedisptsStatsMedian
globDedisptsStatsRatio0
globDedisptsStatsPosPct
globDedisptsStatsNegCnt
globDedisptsStatsRatio1
globDedisptsStatsMean
windTimeStatsStd0
windTimeStatsStd1
windTimeStatsStd2
windTimeStatsStd3
windTimeStatsStd4
windTimeStatsStd5
windTimeStatsStd6
windTimeStatsStd7
windTimeStatsStd8
windTimeStatsStd9
windTimeStatsStd10
windTimeStatsStd11
windTimeStatsStd12
windTimeStatsStd13
windTimeStatsStd14
windTimeStatsStd15
w

In [6]:
dfSubLabel = df[df['Label']==labelID]

In [15]:
dfSubLabel.to_csv('filterbanksLbl%i.csv'%labelID, columns=['filterbank','Buffer'], header=False, index=False)

### Generate Extract scripts

In [12]:
headerStr = '#!/bin/bash\n\n'
abc0Str = headerStr
abc1Str = headerStr
abc2Str = headerStr
abc3Str = headerStr

In [13]:
for idx,row in dfSubLabel.iterrows():
    extractStr = '/home/artemis/Survey/Scripts/extractBuffer.rb -b %i /databk/pipeline/processed/%s\n'%(row.Buffer, row.filterbank)
    
    fbPrefix = row.filterbank.split('.fil')[0]
    cpImgStr = 'cp /databk/pipeline/output/%s.buffer%i.d%i.png .\n'%(fbPrefix, row.Buffer, row.BinFactor)
    
    if row.Beam == 0 or row.Beam==1: abc0Str += extractStr + cpImgStr
    elif row.Beam == 2 or row.Beam==3: abc1Str += extractStr + cpImgStr
    elif row.Beam == 4 or row.Beam==5: abc2Str += extractStr + cpImgStr
    elif row.Beam == 6: abc3Str += extractStr + cpImgStr

In [14]:
open('extractLbl%iabc0.sh'%labelID,'w').write(abc0Str)
open('extractLbl%iabc1.sh'%labelID,'w').write(abc1Str)
open('extractLbl%iabc2.sh'%labelID,'w').write(abc2Str)
open('extractLbl%iabc3.sh'%labelID,'w').write(abc3Str)